<a href="https://colab.research.google.com/github/stuart-lane/MachineLearning/blob/main/NewSimulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random
import math

import scipy
from scipy.stats import norm as normal
import sklearn

from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import torch
from torch.optim import Adam
from torch.optim import LBFGS

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

np.random.seed(123456)

DATA GENERATING PROCESS

In [ ]:
n = 1000

beta0 = 0
beta1 = 1

x = np.random.randn(n)
u = np.random.logistic(0, 1, n)

ystar = beta0 + beta1 * x + u
y = (ystar > 0).astype(int)

u = u.reshape(-1, 1)
x = np.hstack((np.ones((n, 1)), x.reshape(-1, 1)))
y = y.reshape(-1, 1)

LOG-LIKELIHOOD FUNCTION

In [ ]:
def negative_log_likelihood(beta, X, y):
    if isinstance(X, np.ndarray):
        xbhat = np.dot(X, beta)
        yhat = 1 / (1 + np.exp(-xbhat))
        loss = -np.sum(y * np.log(yhat) + (1 - y) * np.log(1 - yhat))
    elif isinstance(X, torch.Tensor):
        xbhat = torch.mm(X, beta)
        yhat = torch.sigmoid(xbhat)
        loss = -torch.sum(y * torch.log(yhat) + (1 - y) * torch.log(1 - yhat))
    else:
        raise ValueError("Unsupported input type. Use either numpy array or torch tensor.")
    return loss

DATA GENERATING PROCESS

In [2]:
n = 1000

beta0 = 0
beta1 = 1
sigma = 1
beta = np.array([beta0, beta1])

x = np.random.randn(n)
u = np.random.normal(0, sigma, n)

y_cutoff = 0

ystar = beta0 + beta1 * x + u
yind = (ystar > y_cutoff).astype(int)
y = yind * ystar

u = u.reshape(-1, 1)
x = np.hstack((np.ones((n, 1)), x.reshape(-1, 1)))
y = y.reshape(-1, 1)

LOG-LIKELIHOOD FUNCTION

In [19]:
def normal_pdf(X, mu, sigma = 1):
    if isinstance(X, np.ndarray):
        constant = 1.0 / np.sqrt(2 * math.pi * sigma**2)
        exponent = -((X - mu)**2) / (2 * sigma**2)
        return constant * math.exp(exponent)
    elif isinstance(X, torch.Tensor):
        constant = 1.0 / torch.sqrt(2 * torch.tensor(math.pi) * sigma**2)
        exponent = -((X - mu)**2) / (2 * sigma**2)
        return constant * torch.exp(exponent)
    else:
        raise ValueError("Unsupported input type. Use either numpy array or torch tensor.")
    return loss


def negative_log_likelihood(beta, X, y, yind, y_cutoff, sigma = 1):
    if isinstance(X, np.ndarray):
        xbeta = np.dot(X, beta)
        term1 = (y - xbeta) / sigma
        normpdf = normal_pdf(term1, 0, 1) / sigma
        cdf_term2 = normal.cdf((xbeta - y_cutoff) / sigma)
        loss = - (np.sum(yind * np.log(normpdf) + (1 - yind) * np.log(1 - (cdf_term2))))
    elif isinstance(X, torch.Tensor):
        xbeta = torch.matmul(X, beta)
        term1 = (y - xbeta) / sigma
        normpdf = normal_pdf(term1, 0, 1) / sigma
        cdf_term2 = (xbeta - y_cutoff) / sigma
        loss = - (torch.sum(yind * torch.log(normpdf) + (1 - yind) * torch.log(1 - cdf_term2)))
    else:
        raise ValueError("Unsupported input type. Use either numpy array or torch tensor.")
    return loss

DATA FORMATTING

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 123)

train_df_keys = {'constant': x_train[:,0].flatten(), 'X1': x_train[:,1].flatten(), 'y': y_train.flatten()}
train_df = pd.DataFrame(train_df_keys)

In [31]:
# Convert the variables to torch tensors
x = torch.tensor(x_train, dtype = torch.float64, requires_grad = True)
y = torch.tensor(y_train, dtype = torch.float64, requires_grad = True)
yi = torch.tensor(yind, dtype = torch.float64, requires_grad = True)
yc = torch.tensor(y_cutoff, dtype = torch.float64, requires_grad = True)
beta = torch.randn(x.shape[1], 1, dtype = torch.float64, requires_grad = True)

print(beta)

tensor([[0.2765],
        [1.8134]], dtype=torch.float64, requires_grad=True)


LOGISTIC REGRESSION WITH SKLEARN

In [ ]:
model = LogisticRegression(solver = 'lbfgs', max_iter = 1000)  # You can change the solver and max_iter as needed
model.fit(x_train, y_train)

intercept = model.intercept_[0]
coefficient = model.coef_[0][1]

In [ ]:
print(f"Intercept: {intercept}")
print(f"Coefficient for 'Regressor': {coefficient}")

Intercept: -0.12587614423279017
Coefficient for 'Regressor': 0.9506309771391525


STANDARD LOGISTIC REGRESSION

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_regression(X, y, learning_rate, num_iterations):
    m, n = X.shape
    theta = np.zeros((n, 1))

    for i in range(num_iterations):
        z = np.dot(X, theta)
        h = sigmoid(z)
        gradient = np.dot(X.T, (h - y)) / m
        theta -= learning_rate * gradient
        if (i+1) % 250 == 0:
          print(f"Iteration: {i+1} || Gradients: {gradient[0][0]} and {gradient[1][0]}")
          print(f"Estimated Parameters (Weights): {theta[0][0]} and {theta[1][0]}")

    return theta

learning_rate = 0.1
num_iterations = 1000

theta = logistic_regression(x_train, y_train, learning_rate, num_iterations)

Iteration: 250 || Gradients: -0.00017192177053207462 and -0.002321160390690911
Estimated Parameters (Weights): 0.061198562140810986 and 0.9318501526068228
Iteration: 500 || Gradients: -3.739478394055376e-06 and -5.142163406040949e-05
Estimated Parameters (Weights): 0.062283941471408026 and 0.9465825388605845
Iteration: 750 || Gradients: -8.340395103632881e-08 and -1.1531930849111027e-06
Estimated Parameters (Weights): 0.06230777791217243 and 0.9469109376872196
Iteration: 1000 || Gradients: -1.868580222913474e-09 and -2.5868296530534572e-08
Estimated Parameters (Weights): 0.06230831039952592 and 0.9469183033956795


AUTOGRAD

In [32]:
# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    dev = "cuda"
else:
    dev = "cpu"

print(f"Running on: {dev}")

# Set up the Adam optimizer
learning_rate = 0.01
optimizer = LBFGS([beta], lr = learning_rate)

#negative_log_likelihood(beta, x, y, yi, yc, sigma = 1)

# Define a function to calculate the loss and update model parameters
def calculate_loss():
    optimizer.zero_grad()
    value = negative_log_likelihood(beta, x, y, yi, yc)
    value.backward()
    return value

num_iter = 10
for i in range(1, num_iter + 1):
  logl = optimizer.step(calculate_loss)
  logl_value = logl.detach().item()
  estimated_weights = beta.detach().numpy()
  if (i + 1) % 25 == 0:
    print(f"Iteration {i + 1} || Log-value {logl_value}")
    print(f"Estimated weights: {estimated_weights}")

estimated_weights_LBFGS = beta.detach().numpy()
print("Estimated Parameters (Weights):", estimated_weights_LBFGS)

Running on: cpu
Iteration 2 || Log-value nan
Estimated weights: [[0.3250124]
 [1.7104483]]
Iteration 3 || Log-value nan
Estimated weights: [[0.36337844]
 [1.61112273]]
Iteration 4 || Log-value nan
Estimated weights: [[0.39406622]
 [1.53201509]]
Iteration 5 || Log-value nan
Estimated weights: [[0.41865421]
 [1.46884336]]
Iteration 6 || Log-value nan
Estimated weights: [[0.43839354]
 [1.41825729]]
Iteration 7 || Log-value nan
Estimated weights: [[0.45474395]
 [1.37590146]]
Iteration 8 || Log-value nan
Estimated weights: [[0.46746403]
 [1.34339297]]
Iteration 9 || Log-value nan
Estimated weights: [[0.47774081]
 [1.31724737]]
Iteration 10 || Log-value nan
Estimated weights: [[0.48605541]
 [1.29613295]]
Iteration 11 || Log-value nan
Estimated weights: [[0.49279343]
 [1.27904007]]
Estimated Parameters (Weights): [[0.49279343]
 [1.27904007]]


NEURAL NETWORK LOGIT

In [ ]:
class LogitModel(nn.Module):
    def __init__(self, input_dim):
        super(LogitModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)  # Single output unit

    def forward(self, x):
        x = x.to(self.linear.weight.dtype)  # Cast input to the same data type as the model's parameters
        x = self.linear(x)
        x = torch.sigmoid(x)  # Apply sigmoid activation for binary classification
        return x

input_dim = x.shape[1]

model = LogitModel(input_dim)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

epochs = 5000

# Training loop
for epoch in range(epochs+1):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Zero the gradients

    outputs = model(x)
    outputs = outputs.float()
    y = y.float()

    loss = loss_function(outputs, y)
    loss.backward()
    optimizer.step()
    if epoch % (epochs / 10) == 0:
        print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}")

# View the learned parameters
print("Learned parameters:")
print(f"Model weights: {model.linear.weight[0][0]}, {model.linear.weight[0][1]}") # these are the parameters of interest
print(f"Model bias:{model.linear.bias[0]}")

Epoch [0/5000], Loss: 0.6721
Epoch [500/5000], Loss: 0.6202
Epoch [1000/5000], Loss: 0.6087
Epoch [1500/5000], Loss: 0.6069
Epoch [2000/5000], Loss: 0.6067
Epoch [2500/5000], Loss: 0.6067
Epoch [3000/5000], Loss: 0.6067
Epoch [3500/5000], Loss: 0.6067
Epoch [4000/5000], Loss: 0.6067
Epoch [4500/5000], Loss: 0.6067
Epoch [5000/5000], Loss: 0.6067
Learned parameters:
Model weights: -0.3844618499279022, 0.9469168186187744
Model bias:0.4467701315879822
